In [18]:
import sys
import os
import numpy as np
import pandas as pd
import nibabel as nb
import torch
import matplotlib.pyplot as plt
sys.path.append('/host/d/Github')
import Whole_heart_segmentation_junzhe.functions_collection as ff
import Whole_heart_segmentation_junzhe.Data_processing as Data_processing
import Whole_heart_segmentation_junzhe.Build_lists.Build_list as Build_list
import Whole_heart_segmentation_junzhe.data_loader.random_aug as random_aug
import Whole_heart_segmentation_junzhe.data_loader.generator as generator
from torch.utils.data import Dataset, DataLoader


### step 1: define trial name

In [12]:
trial_name = 'trial'

### step 2: build patient list

In [13]:
# change the excel path to your own path
patient_list_spreadsheet = os.path.join('/host/d/Github/Whole_heart_segmentation_junzhe/example_data/Patient_list','patient_list.xlsx')
build_sheet =  Build_list.Build(patient_list_spreadsheet)
# train
_, patient_id_list_train, slice_index_list_train, img_file_list_train, seg_file_list_train = build_sheet.__build__(batch_list = [0])  # just as an example, use batch 0 for train
print('all train img files:', img_file_list_train)

# # define val
# _,_,input_file_val, reference_file_val = build_sheet.__build__(batch_list = [1])  # just as an example, use the same batch for val



all train img files: ['/host/d/GitHub/Whole_heart_segmentation_junzhe/example_data/data/ID_0002/img/slice_0.nii.gz'
 '/host/d/GitHub/Whole_heart_segmentation_junzhe/example_data/data/ID_0002/img/slice_1.nii.gz'
 '/host/d/GitHub/Whole_heart_segmentation_junzhe/example_data/data/ID_0002/img/slice_2.nii.gz'
 '/host/d/GitHub/Whole_heart_segmentation_junzhe/example_data/data/ID_0002/img/slice_3.nii.gz'
 '/host/d/GitHub/Whole_heart_segmentation_junzhe/example_data/data/ID_0002/img/slice_4.nii.gz'
 '/host/d/GitHub/Whole_heart_segmentation_junzhe/example_data/data/ID_0002/img/slice_5.nii.gz']


### data generator

In [14]:
# define this generator
generator_train = generator.Dataset_CMR(
    image_file_list = img_file_list_train,
    
    seg_file_list = seg_file_list_train,

    image_shape = [128,128],
    center_crop_according_to_which_class  = [1], #default: crop according to class 1 (LV)

    shuffle = True,
    image_normalization = True,
    augment = True,
    augment_frequency = 0.1,
    )

In [15]:
ds = generator_train
dl = DataLoader(ds, batch_size = 3, shuffle = False, pin_memory = True, num_workers = 0)# cpu_count())

In [25]:
# build model
import argparse
import os

def get_args_parser( vit_type = "vit_h"):
    parser = argparse.ArgumentParser('SAM fine-tuning', add_help=True)

    parser.add_argument('--resume', default = 'weights')

    parser.add_argument('--img_size', default=128, type=int) 

    parser.add_argument('--vit_type', default=vit_type, type=str, choices=[ 'vit_b', 'vit_l', 'vit_h'],)
    
 
    return parser

import Whole_heart_segmentation_junzhe.segment_anything.model as model
args = get_args_parser(vit_type = "vit_h")
args = args.parse_args([])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.build_model(args, device)

FileNotFoundError: [Errno 2] No such file or directory: 'weights'

In [ ]:


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for batch in dl:
    dictionary = batch
    images = dictionary['image']
    segs = dictionary['mask']
    
    # put into GPU if needed
    images = images.to(device)
    segs = segs.to(device)

    print('shape of images:', images.shape, ' shape of segs:', segs.shape)

    output = model(images)


    

loading image file: /host/d/GitHub/Whole_heart_segmentation_junzhe/example_data/data/ID_0002/img/slice_3.nii.gz  seg file: /host/d/GitHub/Whole_heart_segmentation_junzhe/example_data/data/ID_0002/seg/slice_3.nii.gz
after augmentation, image min: 0.0  max: 1.0
loading image file: /host/d/GitHub/Whole_heart_segmentation_junzhe/example_data/data/ID_0002/img/slice_0.nii.gz  seg file: /host/d/GitHub/Whole_heart_segmentation_junzhe/example_data/data/ID_0002/seg/slice_0.nii.gz
after augmentation, image min: 0.0  max: 1.0
loading image file: /host/d/GitHub/Whole_heart_segmentation_junzhe/example_data/data/ID_0002/img/slice_5.nii.gz  seg file: /host/d/GitHub/Whole_heart_segmentation_junzhe/example_data/data/ID_0002/seg/slice_5.nii.gz
after augmentation, image min: 0.0  max: 1.0
shape of images: torch.Size([3, 1, 128, 128])  shape of segs: torch.Size([3, 1, 128, 128])
loading image file: /host/d/GitHub/Whole_heart_segmentation_junzhe/example_data/data/ID_0002/img/slice_1.nii.gz  seg file: /host/